# Spark Data Frames

Spark SQL is a Spark module for structured data processing. Unlike the basic Spark RDD API, the interfaces provided by Spark SQL provide Spark with more information about the structure of both the data and the computation being performed. Internally, Spark SQL uses this extra information to perform extra optimizations.

There are several ways to interact with Spark SQL including SQL, the DataFrames API and the Datasets API.
When computing a result the same execution engine is used, independent of which API/language you are using to express the computation. This unification means that developers can easily switch back and forth between the various APIs based on which provides the most natural way to express a given transformation.

# Setup

In [1]:
import os, sys
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

## Setup Spark

In [9]:
# %load ../01_Distributed_Computing_HDFS_Distributed_Data_Sets/pyspark_init_arc.py
#
# This configuration works for Spark on arc.insight.gsu.edu
#
import os, sys
# set OS environment variable
os.environ["SPARK_HOME"] = '/usr/hdp/2.4.2.0-258/spark'
# add Spark library to Python
sys.path.insert(0, os.path.join(os.environ["SPARK_HOME"], 'python'))

# import package
import pyspark
from pyspark.context import SparkContext, SparkConf

import atexit
def stop_my_spark():
    sc.stop()
    del(sc)

# Register exit    
atexit.register(stop_my_spark)

# Configure and start Spark ... but only once.
if not 'sc' in globals():
    conf = SparkConf()
    conf.setAppName('MyFirstSpark') ## you may want to change this
    conf.setMaster('yarn-client')
    conf.set('spark.ui.port', '%d'%(52000+np.int(np.random.rand(1)*10000)))
    sc = SparkContext(conf=conf)
    print "Launched Spark version %s with ID %s" % (sc.version, sc.applicationId)
    print "http://arc.insight.gsu.edu:8088/cluster/app/%s"% (sc.applicationId)


Launched Spark version 1.6.1 with ID application_1508160140652_0028
http://arc.insight.gsu.edu:8088/cluster/app/application_1508160140652_0028


## Add SQL Context and a couple of classes

In [11]:
from pyspark.sql import SQLContext, Row, DataFrame
sqlCtx = SQLContext(sc)

In [12]:
user_df = sqlCtx.read.json('/data/yelp/user')
user_df.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- compliments: struct (nullable = true)
 |    |-- cool: long (nullable = true)
 |    |-- cute: long (nullable = true)
 |    |-- funny: long (nullable = true)
 |    |-- hot: long (nullable = true)
 |    |-- list: long (nullable = true)
 |    |-- more: long (nullable = true)
 |    |-- note: long (nullable = true)
 |    |-- photos: long (nullable = true)
 |    |-- plain: long (nullable = true)
 |    |-- profile: long (nullable = true)
 |    |-- writer: long (nullable = true)
 |-- elite: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- fans: long (nullable = true)
 |-- friends: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- type: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- votes: struct (nullable = true)
 |    |-- cool: long (nullable = true)
 |    |-- funny: long (nullable = true)

How many records?

In [13]:
user_df.count()

68655

In [15]:
sqlCtx.read.json()
user_df.select('name', 'average_stars', 'compliments')

+---------+-------------+--------------------+
|     name|average_stars|         compliments|
+---------+-------------+--------------------+
|   Russel|         4.14|[76,15,11,48,null...|
|   Jeremy|         3.67|[1675,204,594,111...|
|  Michael|         3.68|[134,22,96,99,4,1...|
|      Ken|         4.64|[null,2,null,null...|
|Katherine|          3.8|[1,2,null,1,null,...|
|     Joan|         4.35|[3712,273,1527,27...|
|   George|         4.45|[13,4,2,8,1,1,5,n...|
|     Mary|         3.88|[1,null,null,1,nu...|
|    Nader|         3.59|[5,1,8,5,null,2,3...|
|   Kiffen|          5.0|[null,null,null,1...|
|   Joseph|          4.3|[2,null,null,1,nu...|
|    Helen|         4.01|[213,82,70,265,3,...|
|   Yishan|         3.83|[1,2,1,null,null,...|
|    David|         3.69|[1,null,2,null,nu...|
|   Angela|         4.92|[null,null,null,n...|
| Michelle|          3.6|[null,null,null,n...|
|  Jeffrey|         1.33|[null,null,null,n...|
|    Lynda|         2.42|[null,null,null,n...|
|  Heather|  

In [38]:
user_df.registerTempTable('users')
sqlCtx.sql("SELECT name, average_stars, compliments FROM users WHERE average_stars > 4").show()

+---------+-------------+--------------------+
|     name|average_stars|         compliments|
+---------+-------------+--------------------+
|   Russel|         4.14|[76,15,11,48,null...|
|      Ken|         4.64|[null,2,null,null...|
|     Joan|         4.35|[3712,273,1527,27...|
|   George|         4.45|[13,4,2,8,1,1,5,n...|
|   Kiffen|          5.0|[null,null,null,1...|
|   Joseph|          4.3|[2,null,null,1,nu...|
|    Helen|         4.01|[213,82,70,265,3,...|
|   Angela|         4.92|[null,null,null,n...|
|  Heather|         4.38|[1,null,null,null...|
|  Anthony|         4.19|[null,null,null,n...|
|Stephanie|          5.0|[null,null,null,n...|
|     Maya|         4.56|[null,null,null,n...|
|   Stacey|         4.43|[null,null,null,n...|
|     Cole|          5.0|[null,null,null,n...|
|   Angela|          5.0|[null,null,null,n...|
|LaGabriel|         4.38|[null,null,null,n...|
| Sunshine|          4.5|[null,null,null,n...|
|    Sarah|         4.07|[null,null,null,n...|
|        K|  

In [40]:
review_df = sqlCtx.read.json('/data/yelp/review')
review_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- text: string (nullable = true)
 |-- type: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- votes: struct (nullable = true)
 |    |-- cool: long (nullable = true)
 |    |-- funny: long (nullable = true)
 |    |-- useful: long (nullable = true)



In [41]:
review_df.registerTempTable('reviews')

In [58]:
jnt_df = sqlCtx.sql("""
SELECT business_id, AVG(stars) AS Mstars, VARIANCE(stars) AS Vstars, COUNT(*) AS n FROM users
JOIN reviews 
ON users.user_id=reviews.user_id
GROUP BY business_id
HAVING COUNT(*)>20
""")
jnt_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- Mstars: double (nullable = true)
 |-- Vstars: double (nullable = false)
 |-- n: long (nullable = false)



In [59]:
jnt_df.sort('n', ascending=).show()

+--------------------+------------------+------------------+---+
|         business_id|            Mstars|            Vstars|  n|
+--------------------+------------------+------------------+---+
|GfUJJXI7bzTRaOgEZ...| 3.761904761904762|1.3904761904761904| 21|
|MWoM_zf75KD0VcOm8...|3.9047619047619047|1.7904761904761906| 21|
|EXtCgZoxHNjXrqPCF...| 4.285714285714286| 0.914285714285714| 21|
|ishjLG3Sc94ZOelXZ...| 3.238095238095238| 1.690476190476191| 21|
|C7hN8QQUxC8uQk2Yw...|3.2857142857142856| 2.414285714285714| 21|
|2G-I9lROOVwkIkBSE...|3.7142857142857144|1.1142857142857143| 21|
|5tey93wPQOb7z6jOL...|3.2857142857142856|2.0142857142857142| 21|
|0WI0ydH2HtFxFswYX...| 3.619047619047619|2.5476190476190474| 21|
|KZfhb_wZj1qqKMJIN...| 3.761904761904762|0.7904761904761906| 21|
|93fEJZYc3B-jOMaXc...|               4.0|               1.3| 21|
|rE0-KhjcOQOWIaiyb...|3.4761904761904763|1.2619047619047619| 21|
|LT00Q44xHSFTs1wJP...| 4.285714285714286|0.7142857142857143| 21|
|5iYdwupq9xiko0PCU...| 4.